
# Fuzzy Logic
## Assignment 2 - Introduction to Artificial Intelligence
### Student ID: 1301184521
### Name : Lalu M. Riza Rizky
### Class : IF-42-INT

You are given Mahasiswa.xls containing data of 100 students with two attributes: Penghasilan (Monthly income, real number) and Pengeluaran (Monthly expenses, real number) of a family and the unit is millions IDR. Build a fuzzy logic based system to select 20 students who are eligible for registration fee assistance of 50%. The system reads the input file Mahasiswa.xls and outputs a Bantuan.xls file which contains one column vector containing 20 rows of integer-valued numbers (whole numbers) that represent the row / record numbers (1-100) in the Mahasiswa.xls file.

#### Things you must observe:
* Number and Linguistic Names for each input
* Shape and Range of Input Membership Functions
* Inference Rules
* Defuzzification Method
* Shape dan Range of Output Membership Functions (as appropriate with your chosen defuzzification method)

#### Processes you need to build (can be functions/procedures):
* Read file
* Fuzzification
* Inference
* Defuzzification

Output from your sistem a Bantuan.xls file which contains one column vector containing 20 rows of integer-valued numbers (whole numbers) that represent the row / record numbers (1-100) in the Mahasiswa.xls file.

#### Program Specification
* Language: Python, with Jupyter Notebook
* Library: pandas
* Membership Function Type: Trapezoidal
* Total Fuzzy Rule: 9
* Defuzzification Method: Constant Defuzzification (Sugeno)

#### Note
This is a simplified version of my previous code written in R. This code only goes through the calculation and giving the result, without giving any visualization and detailed explaination at all. Also the defuzzification method implemented in this code is different compared to the R version where the R version implements center of gravity (Mamdani) method, this version implements constant defuzzification (Sugeno) method.

To read the version with more detailed explaination, please check: https://github.com/zaRizk7/fuzzy-logic/blob/main/fuzzy-logic.ipynb

#### Step Rundown
Here are the steps of the structure building of this program:

##### 1. Import required libraries
It is required to import pandas for doing this task in Python since Python does not directly have a way to read and write directly .xlsx or .csv file.

In [1]:
import pandas as pd

##### 2. Setting up URL for dataset and inference rule table
Since the code is originally in Google Colab. In order to make it easier to import the tables required. It has been setup with URLs which directly linked to Google Spreadsheet and exports the spreadsheet to a .xlsx file.

In [2]:
dataset_url = 'https://docs.google.com/spreadsheets/d/1tojklCVqyH9b4_hJMxPp-3zpSPS4fkPLknvuBHpa5WY/export?format=xlsx'
ruleset_url = 'https://docs.google.com/spreadsheets/d/1VqhacFI-OIXBn7XxVakAorcVLJIFXAFgmn3k_NMVwms/export?format=xlsx'

##### 3. Determining the membership function
The membership function is using the trapedoizal based version. To make sure that the definition of membership function is easier, create a base function for the membership function. There are three versions defined in this program:
* Trapezoidal
* Trapezoidal Ascend
* Trapezoidal Descend

In [3]:
class Membership:
  def trapezoid(self, a, b, c, d):
    def membership(x):
      if x <= a or x > d:
        return 0
      elif a < x < b:
        return (x - a) / (b - a)
      elif b <= x <= c:
        return 1
      else:
        return -(x - d) / (d - c)
    return membership

  def trapezoid_ascend(self, a, b):
    def membership(x):
      if x <= a:
        return 0
      elif a < x < b:
        return (x - a) / (b - a)
      else:
        return 1
    return membership

  def trapezoid_descend(self, a, b):
    def membership(x):
      if x <= a:
        return 1
      elif a < x < b:
        return (b - x) / (b - a)
      else:
        return 0
    return membership

Next, determine the membership range for income, expense, and result. Based on the data observation, the defined ranges for the membership function of each input are:


Income:
* Low Membership:
  * Minimum: 5
  * Maximum: 7
* Average Membership:
  * Lower Minimum: 6
  * Lower Maximum: 14
  * Upper Minimum: 8
  * Upper Maximum: 12
* High Membership:
  * Minimum: 11
  * Maximum: 16

In [4]:
class Income(Membership):
  def __init__(self, value):
    self.value = value
    self.membership_value = {
        'low': self.low(),
        'average': self.average(),
        'high': self.high()
    }

  def membership(self, label):
    return self.membership_value[label]

  def low(self):
    membership = self.trapezoid_descend(7, 9)
    return membership(self.value)

  def average(self):
    membership = self.trapezoid(6, 8, 12, 14)
    return membership(self.value)
  
  def high(self):
    membership = self.trapezoid_ascend(11, 16)
    return membership(self.value)
    


Expense:
* Low Membership:
  * Minimum: 4
  * Maximum: 6
* Average Membership:
  * Lower Minimum: 5
  * Lower Maximum: 9
  * Upper Minimum: 6
  * Upper Maximum: 8
* High Membership:
  * Minimum: 8
  * Maximum: 10

In [5]:
class Expense(Membership):
  def __init__(self, value):
    self.value = value
    self.membership_value = {
        'low': self.low(),
        'average': self.average(),
        'high': self.high()
    }

  def membership(self, label):
    return self.membership_value[label]

  def low(self):
    membership = self.trapezoid_descend(4, 6)
    return membership(self.value)

  def average(self):
    membership = self.trapezoid(5, 6, 8, 9)
    return membership(self.value)
  
  def high(self):
    membership = self.trapezoid_ascend(8, 10)
    return membership(self.value)
    

##### 4. Inference, Defuzzification, Output

###### 4.1. Inference
Firstly, the fuzzy values are calculated both for income, expense, and the result. Result fuzzy values are based on the minimum value between income and expense fuzzy values. The inferred values are based on the maximum value between the result fuzzy values between the defined labels for each data. So, to infer the entire dataset will be required to iterate each row to calculate each data inferred values.

###### 4.2. Defuzzification
The constant defuzzification (Sugeno) method utilizes constant values defined by each label used for the result. The formula used to calculate the crisp value is, $z^*=\frac{\sum_{i = 1}^{n}\mu B_i c_i}{\sum_{i = 1}^{n}\mu B_i}$ where $z^*$ is the crisp value, $c_i$ is the constant for the $i^{th}$ linguistic / label, and $\mu B_i$ is the membership for the $i^{th}$ lingustic / label.

In [6]:
class AssignmentFuzzySystem:
  def __init__(self, dataset_url, ruleset_url):
    self.dataset = pd.read_excel(dataset_url)
    self.ruleset = pd.read_excel(ruleset_url)
    self.result = self.calculate(20)
    self.result.columns = ['id']

  def infer_data(self, data):
    income = lambda label: Income(data['Penghasilan']).membership(label)
    expense = lambda label: Expense(data['Pengeluaran']).membership(label)
    label = ['reject', 'consider', 'accept']
    membership = [0, 0, 0]
    for idx, rule in self.ruleset.iterrows():
      value = min(income(rule['income']), expense(rule['expense']))
      label_idx = label.index(rule['result'])
      membership[label_idx] = max(membership[label_idx], value)
    return membership

  def infer_dataset(self):
    values = []
    for idx, data in self.dataset.iterrows():
      values.append(self.infer_data(data))
    return values

  def defuzzification(self):
    values = []
    constant = [60, 80, 100]
    for id, val in zip(range(1, 101), self.infer_dataset()):
      x_multiple = sum([x * c for x, c in zip(constant, val)])
      x_sum = sum(val)
      crisp_value = x_multiple / x_sum
      values.append({'id': id, 'value': crisp_value})
    return values

  def calculate(self, n):
    result = self.defuzzification()
    result = sorted(result, key=lambda x: x['value'], reverse=True)
    result = [x['id'] for x in result]
    result = pd.DataFrame(result)
    result = result.head(n)
    return result

  def export(self):
    self.result.to_csv('bantuan.csv', index=False)
    self.result.to_excel('bantuan.xlsx', index=False)


###### 4.3. Output
In order to output result of the process done by the fuzzy system, since the use of OOP in the program. It needs to construct the object with the given URL for dataset and inference rule. While constructing, it will do a calculation to infer the fuzzy values for each label / linguistic defined. After the object successfully constructed, it will output the result and export it directly as a .csv and .xlsx file.

In [7]:
fuzzy_system = AssignmentFuzzySystem(dataset_url, ruleset_url)

In [8]:
fuzzy_system.result

,id
0,4
1,7
2,11
3,13
4,33
5,38
6,41
7,46
8,47
9,49


In [9]:
fuzzy_system.export()